In [ ]:
pip install pyspark

In [2]:
from pyspark import SparkConf, SparkContext

In [3]:
conf = SparkConf().setAppName("practical session 2").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [4]:
#part 4.1 matrix operations
V = sc.textFile("/content/drive/MyDrive/Colab Notebooks/Semester 3/BigData/part of B. Jeudy/Practical_Session_2/data/V.txt")
W = sc.textFile("/content/drive/MyDrive/Colab Notebooks/Semester 3/BigData/part of B. Jeudy/Practical_Session_2/data/W.txt")

In [18]:
V1 = V.map(lambda x:(x.split()[0],float(x.split()[1])))

In [19]:
W1 = W.map(lambda x:(x.split()[0],float(x.split()[1])))

In [26]:
V1.join(W1).map(lambda x:(x[0],x[1][0]*x[1][1])).collect()

[('3', 2.9899999999999998), ('1', -2.25), ('2', 10.0)]

In [44]:
#4.2
from math import sqrt
normV = sqrt(V.map(lambda x:(x.split()[0],float(x.split()[1]))).map(lambda x:(x[0],x[1]*x[1])).values().sum())
normW = sqrt(W.map(lambda x:(x.split()[0],float(x.split()[1]))).map(lambda x:(x[0],x[1]*x[1])).values().sum())

In [142]:
normW

3.9420806688854046

In [46]:
#4.3
M = sc.textFile("/content/drive/MyDrive/Colab Notebooks/Semester 3/BigData/part of B. Jeudy/Practical_Session_2/data/M.txt")

In [52]:
M1 = M.map(lambda x:((x.split()[0],x.split()[1]),float(x.split()[2])))

In [100]:
M1.collect()

[(('1', '1'), 3.0),
 (('1', '2'), 2.0),
 (('1', '3'), 7.0),
 (('1', '4'), 2.0),
 (('2', '2'), 7.0),
 (('2', '3'), 1.0),
 (('3', '3'), 1.0)]

In [104]:
V2 = V1.map(lambda x:(("1",x[0]),x[1])).union(V1.map(lambda x:(("2",x[0]),x[1]))).union(V1.map(lambda x:(("3",x[0]),x[1])))

In [103]:
V1.collect()

[('1', 1.5), ('2', 5.0), ('3', 1.3)]

In [106]:
rdd_mv = M1.join(V2)

In [107]:
sorted(rdd_mv.collect())

[(('1', '1'), (3.0, 1.5)),
 (('1', '2'), (2.0, 5.0)),
 (('1', '3'), (7.0, 1.3)),
 (('2', '2'), (7.0, 5.0)),
 (('2', '3'), (1.0, 1.3)),
 (('3', '3'), (1.0, 1.3))]

In [111]:
#4.4
rdd_mv.map(lambda x:(x[0],x[1][0]*x[1][1])).map(lambda x:(x[0][0],x[1])).reduceByKey(lambda x,y:x+y).collect()

[('1', 23.6), ('3', 1.3), ('2', 36.3)]

In [133]:
#4.5
V0 = sc.parallelize([("1",1),("2",1),("3",1),("4",1)])
normV0 = sqrt(V0.map(lambda x:(x[0],x[1]*x[1])).values().sum())
NV0 = V0.map(lambda x:(x[0],x[1]/normV0))

In [177]:
def update(NV):
  NVtemp = NV.map(lambda x:(("1",x[0]),x[1])).union(NV.map(lambda x:(("2",x[0]),x[1]))).union(NV.map(lambda x:(("3",x[0]),x[1]))).union(NV.map(lambda x:(("4",x[0]),x[1])))
  VV = M1.join(NVtemp).map(lambda x:(x[0],x[1][0]*x[1][1])).map(lambda x:(x[0][0],x[1])).reduceByKey(lambda x,y:x+y)
  normVV = sqrt(VV.map(lambda x:(x[0],x[1]*x[1])).values().sum())
  NVV = VV.map(lambda x:(x[0],x[1]/normVV))

  l = VV.join(NVV).map(lambda x:(x[0],x[1][0]*x[1][1]))
  return l,NVV

In [ ]:
NV = NV0
for i in range(10):
  l,newNV = update(NV)
  print("Iteration "+str(i), sorted(l.collect()))
  NV = newNV

Iteration 0 [('1', 6.066047713783695), ('2', 1.9807502738885534), ('3', 0.030949223029508647)]
Iteration 1 [('1', 3.0248292300113913), ('2', 2.326075779480949), ('3', 0.0007159359124287314)]
Iteration 2 [('1', 2.265694532276658), ('2', 3.630338939715843), ('3', 2.2689618373224017e-05)]
Iteration 3 [('1', 1.8279480400581216), ('2', 4.656244988604974), ('3', 5.934848061486908e-07)]
Iteration 4 [('1', 1.5954876725117664), ('2', 5.187778107996315), ('3', 1.349319496691538e-08)]
Iteration 5 [('1', 1.4859859686890013), ('2', 5.423637004090767), ('3', 2.8788670591956333e-10)]


KeyboardInterrupt: ignored